In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Dropout,
                                     Flatten, Dense, BatchNormalization)

# Define model architecture
def build_emotion_model(input_shape=(48, 48, 1), num_classes=7):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))

    return model

# Build and compile
model = build_emotion_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 46, 46, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 46, 46, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 23, 23, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 21, 21, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 21, 21, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 10, 10, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 8, 8, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 8, 8, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 4, 4, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │         262,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 356,743 (1.36 MB)

 Trainable params: 356,295 (1.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = 'data/train'
val_dir = 'data/test'

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'
)

# Training
model.fit(train_generator, validation_data=val_generator, epochs=25)

# Save the model
model.save('Emotion_Detection.h5')


Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 315s 696ms/step - accuracy: 0.2369 - loss: 2.0003 - val_accuracy: 0.2453 - val_loss: 1.9840
Epoch 2/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.3300 - loss: 1.6627 - val_accuracy: 0.4211 - val_loss: 1.5024
Epoch 3/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.3774 - loss: 1.5669 - val_accuracy: 0.4586 - val_loss: 1.4252
Epoch 4/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.4125 - loss: 1.4956 - val_accuracy: 0.4734 - val_loss: 1.4090
Epoch 5/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.4348 - loss: 1.4442 - val_accuracy: 0.4794 - val_loss: 1.3806
Epoch 6/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 28s 62ms/step - accuracy: 0.4528 - loss: 1.4097 - val_accuracy: 0.4918 - val_loss: 1.3118
Epoch 7/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.4676 - loss: 1.3712 - val_accuracy: 0.4563 - val_loss: 1.3951
Epo

In [5]:
model.fit(train_generator, validation_data=val_generator, epochs=25)

Epoch 1/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6010 - loss: 1.0575 - val_accuracy: 0.5802 - val_loss: 1.1102
Epoch 2/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6028 - loss: 1.0427 - val_accuracy: 0.5890 - val_loss: 1.1188
Epoch 3/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6064 - loss: 1.0413 - val_accuracy: 0.5808 - val_loss: 1.1612
Epoch 4/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6109 - loss: 1.0260 - val_accuracy: 0.5805 - val_loss: 1.1366
Epoch 5/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6142 - loss: 1.0101 - val_accuracy: 0.5712 - val_loss: 1.1586
Epoch 6/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6218 - loss: 1.0026 - val_accuracy: 0.5517 - val_loss: 1.2156
Epoch 7/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6286 - loss: 0.9849 - val_accuracy: 0.5982 - val_loss: 1.1082
Epoch 8/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6290 - loss: 0.9772 - 

In [6]:
model.fit(train_generator, validation_data=val_generator, epochs=25)

Epoch 1/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6683 - loss: 0.8661 - val_accuracy: 0.6119 - val_loss: 1.1029
Epoch 2/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.6774 - loss: 0.8575 - val_accuracy: 0.5876 - val_loss: 1.1621
Epoch 3/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.6819 - loss: 0.8488 - val_accuracy: 0.5936 - val_loss: 1.1511
Epoch 4/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.6800 - loss: 0.8492 - val_accuracy: 0.6084 - val_loss: 1.1153
Epoch 5/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.6782 - loss: 0.8517 - val_accuracy: 0.5972 - val_loss: 1.1590
Epoch 6/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.6803 - loss: 0.8475 - val_accuracy: 0.6096 - val_loss: 1.1290
Epoch 7/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.6794 - loss: 0.8487 - val_accuracy: 0.5861 - val_loss: 1.1759
Epoch 8/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 24s 54ms/step - accuracy: 0.6831 - loss: 0.8426 - 

In [7]:
model.fit(train_generator, validation_data=val_generator, epochs=25)

Epoch 1/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.7081 - loss: 0.7754 - val_accuracy: 0.6222 - val_loss: 1.1162
Epoch 2/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.7038 - loss: 0.7792 - val_accuracy: 0.6133 - val_loss: 1.1234
Epoch 3/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.7118 - loss: 0.7600 - val_accuracy: 0.6073 - val_loss: 1.1360
Epoch 4/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.7134 - loss: 0.7584 - val_accuracy: 0.6236 - val_loss: 1.0995
Epoch 5/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.7154 - loss: 0.7630 - val_accuracy: 0.6212 - val_loss: 1.1197
Epoch 6/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 55ms/step - accuracy: 0.7110 - loss: 0.7655 - val_accuracy: 0.6251 - val_loss: 1.1134
Epoch 7/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.7175 - loss: 0.7419 - val_accuracy: 0.6000 - val_loss: 1.1516
Epoch 8/25
449/449 ━━━━━━━━━━━━━━━━━━━━ 25s 56ms/step - accuracy: 0.7236 - loss: 0.7426 - 

In [8]:
model.fit(train_generator, validation_data=val_generator, epochs=50)

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 59ms/step - accuracy: 0.7399 - loss: 0.6920 - val_accuracy: 0.6145 - val_loss: 1.1196
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 57ms/step - accuracy: 0.7345 - loss: 0.7043 - val_accuracy: 0.6201 - val_loss: 1.1126
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 57ms/step - accuracy: 0.7404 - loss: 0.7034 - val_accuracy: 0.6215 - val_loss: 1.1069
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 59ms/step - accuracy: 0.7410 - loss: 0.6996 - val_accuracy: 0.6198 - val_loss: 1.1160
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.7384 - loss: 0.7017 - val_accuracy: 0.6016 - val_loss: 1.1510
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.7351 - loss: 0.7040 - val_accuracy: 0.6025 - val_loss: 1.1734
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 58ms/step - accuracy: 0.7374 - loss: 0.6978 - val_accuracy: 0.6149 - val_loss: 1.1203
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 26s 59ms/step - accuracy: 0.7438 - loss: 0.6885 - 

In [9]:
model.fit(train_generator, validation_data=val_generator, epochs=50)

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 33s 74ms/step - accuracy: 0.7560 - loss: 0.6487 - val_accuracy: 0.6282 - val_loss: 1.1793
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 71ms/step - accuracy: 0.7661 - loss: 0.6308 - val_accuracy: 0.6188 - val_loss: 1.1737
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 70ms/step - accuracy: 0.7694 - loss: 0.6244 - val_accuracy: 0.6290 - val_loss: 1.2251
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 70ms/step - accuracy: 0.7624 - loss: 0.6409 - val_accuracy: 0.6232 - val_loss: 1.1552
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 33s 73ms/step - accuracy: 0.7664 - loss: 0.6290 - val_accuracy: 0.6318 - val_loss: 1.1449
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 72ms/step - accuracy: 0.7645 - loss: 0.6334 - val_accuracy: 0.6236 - val_loss: 1.1353
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 31s 70ms/step - accuracy: 0.7621 - loss: 0.6411 - val_accuracy: 0.6319 - val_loss: 1.1610
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 71ms/step - accuracy: 0.7652 - loss: 0.6299 - 

In [10]:
model.fit(train_generator, validation_data=val_generator, epochs=50)

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 71ms/step - accuracy: 0.7830 - loss: 0.5879 - val_accuracy: 0.6297 - val_loss: 1.1505
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 31s 69ms/step - accuracy: 0.7805 - loss: 0.5881 - val_accuracy: 0.6321 - val_loss: 1.1576
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 31s 69ms/step - accuracy: 0.7781 - loss: 0.5967 - val_accuracy: 0.6276 - val_loss: 1.1695
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 31s 69ms/step - accuracy: 0.7842 - loss: 0.5867 - val_accuracy: 0.6342 - val_loss: 1.1732
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 31s 69ms/step - accuracy: 0.7766 - loss: 0.6083 - val_accuracy: 0.6351 - val_loss: 1.1775
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 31s 69ms/step - accuracy: 0.7809 - loss: 0.5930 - val_accuracy: 0.6208 - val_loss: 1.2052
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 31s 70ms/step - accuracy: 0.7790 - loss: 0.6022 - val_accuracy: 0.6388 - val_loss: 1.1674
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 72ms/step - accuracy: 0.7813 - loss: 0.5933 - 

In [12]:
model.save("my_model4.keras")

In [13]:
model.save("my_model5.h5")

In [14]:
model.fit(train_generator, validation_data=val_generator, epochs=50)

Epoch 1/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 37s 83ms/step - accuracy: 0.7924 - loss: 0.5638 - val_accuracy: 0.6339 - val_loss: 1.1469
Epoch 2/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 33s 74ms/step - accuracy: 0.7918 - loss: 0.5612 - val_accuracy: 0.6371 - val_loss: 1.1576
Epoch 3/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 72ms/step - accuracy: 0.7963 - loss: 0.5579 - val_accuracy: 0.6225 - val_loss: 1.1924
Epoch 4/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 72ms/step - accuracy: 0.7912 - loss: 0.5698 - val_accuracy: 0.6322 - val_loss: 1.1830
Epoch 5/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 72ms/step - accuracy: 0.7883 - loss: 0.5713 - val_accuracy: 0.6218 - val_loss: 1.1182
Epoch 6/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 33s 72ms/step - accuracy: 0.7884 - loss: 0.5634 - val_accuracy: 0.6308 - val_loss: 1.2058
Epoch 7/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 32s 72ms/step - accuracy: 0.7920 - loss: 0.5624 - val_accuracy: 0.6372 - val_loss: 1.2238
Epoch 8/50
449/449 ━━━━━━━━━━━━━━━━━━━━ 33s 72ms/step - accuracy: 0.7996 - loss: 0.5586 - 